In [ ]:
# 2.2.4에서 crf가 원활하게 구동됨, 코랩으로 구동 시 install뒤 런타임을 재실행해야함

#!pip install git+https://www.github.com/keras-team/keras-contrib.git
#!pip install tensorflow==1.14.0
#!pip install keras==2.2.4
#!pip install tensorflow-gpu==1.14.0


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
# import zipfile

#with zipfile.ZipFile('drive/My Drive/Colab Notebooks/ner_dataset.csv.zip','r') as zip_ref:
#    zip_ref.extractall('drive/My Drive/Colab Notebooks')

In [ ]:
# https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus 에서 nar_dataset.csv.zip을 다운로드
data = pd.read_csv("drive/My Drive/Colab Notebooks/ner_dataset.csv", encoding="latin1")


In [ ]:
# sentence : 문장 idx
# word : 문장을 구성하는 단어
# POS : 형태소
# Tag : BIO
data[:5]

In [ ]:
print(len(data))
# nunique() 유니크의 갯수 추출
print(data['Sentence #'].nunique())
print(data['Word'].nunique())
print(data['Tag'].nunique())

print(data.groupby('Tag').size().reset_index(name='count'))


In [ ]:
# 데이터에 True 가 있는지 확인
print('데이터에 Null 값이 있는지 유무 : ' + str(data.isnull().values.any()))
data.isnull().sum()

In [ ]:
# Sentence # 의 NaN값을 위에 있는 데이터 값으로 채움
data = data.fillna(method = 'ffill')
data['Word'] = data['Word'].str.lower()
print('sentence # 열의 중복을 제거한 값의 개수 : {}'.format(data['Sentence #'].nunique()))
print('Word 열의 중복을 제거한 값의 개수 : {}'.format(data.Word.nunique()))
print('Tag 열의 중복을 제거한 값의 개수 : {}'.format(data.Tag.nunique()))  #BIO 구분의 개수

In [ ]:
# datagram.values는 array형식이기 때문에 list로 변환
# zip을 통해서 하나로 묶은 뒤 단어, BIO 형태를 만듬 ('Stive' , 'B-Per')
func = lambda temp: [(w,t) for w, t in zip(temp["Word"].values.tolist(), temp['Tag'].values.tolist())]

# apply와 groupby를 통해서 Sentence를 기준으로 분류
# 왜 groupby를 통해서 빼지 않았는가  = groupby는 연산이 부가적으로 필요하기 떄문에 
# 변수를 합치는 연산을 생성한 뒤 사용한것
tagged_sentences = [t for t in data.groupby("Sentence #").apply(func)]

In [ ]:
print("전체 샘플 개수: {}".format(len(tagged_sentences)))
ner_tags

In [ ]:
sentences, ner_tags = [], [] 
for tagged_sentence in tagged_sentences: # 47,959개의 문장 샘플을 1개씩 불러온다.
    sentence, tag_info = zip(*tagged_sentence) # 각 샘플에서 단어들은 sentence에 개체명 태깅 정보들은 tag_info에 저장.
 
    sentences.append(list(sentence)) # 각 샘플에서 단어 정보만 저장한다.
    ner_tags.append(list(tag_info)) # 각 샘플에서 개체명 태깅 정보만 저장한다.

In [ ]:
# 40정도의 길이
print('샘플의 최대 길이 : %d' % max(len(l) for l in sentences))
print('샘플의 평균 길이 : %f' % (sum(map(len, sentences))/len(sentences)))
plt.hist([len(s) for s in sentences], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 지정된 단어를 인덱스 1에 지정
src_tokenizer = Tokenizer(oov_token='OOV')
src_tokenizer.fit_on_texts(sentences)
tar_tokenizer = Tokenizer(lower = False)
tar_tokenizer.fit_on_texts(ner_tags)

In [ ]:
src_tokenizer.word_index['OOV']

In [ ]:
# 전체 단어 사이즈
vocab_size = len(src_tokenizer.word_index) + 1
# 전체 테그 사이즈
tag_size = len(tar_tokenizer.word_index) + 1 

In [ ]:
# 정수인코딩
# 1. Tokenizer로 토큰틀 생성
# 2. fit_on_text를 통해서 텍스트에 정수인덱스를 부여(word_index를 통해서 확인가능)
# 3. fit_to_sequences()를 통해 기존 단어를 정수로 변환

x_train = src_tokenizer.texts_to_sequences(sentences)
y_train = tar_tokenizer.texts_to_sequences(ner_tags)

In [ ]:
x_train

In [ ]:
# 예측결과를 문자화할 index를 저장

word_to_index = src_tokenizer.word_index
index_to_word = src_tokenizer.index_word
ner_to_index = tar_tokenizer.word_index
index_to_ner = tar_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [ ]:
decoded = []
for index in x_train[0]:
    decoded.append(index_to_word[index])
print('기존문장 : {}'.format(sentences[0]))
print('디코딩문장 : {}'.format(decoded))

In [ ]:
# 대부분의 문장이 60을 넘지 않음 max = 70 지정
max_len = 70

# 뒤의 부족한 부분은 0으로 채움
x_train = pad_sequences(x_train, padding = 'post', maxlen=max_len)
y_train = pad_sequences(y_train, padding = 'post', maxlen=max_len)

# 학습데이터 분류
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state=777)

In [ ]:
# 정수시퀀스를 array변경
y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [ ]:
y_train

In [ ]:
!pip install seqeval
from keras.callbacks import Callback
from seqeval.metrics import f1_score, classification_report

In [ ]:
# F1-score가 지금까지 중 가장 높은 경우
# o값이 많기 때문에 모델의 성능은 좋게 나올 수 있음
# 하지만 실제 값을 맞추는 것이 아니기 때문에 이를 판단할 기준이 필요하고 이를 f1_score를 통해
# 조화평균을 계산하여 각 요소들의 조화평균값이 가장 높은 모델을 저장함

class F1score(Callback):
    def __init__(self, value = 0.0, use_char=True):
        super(F1score, self).__init__()
        self.value = value
        self.use_char = use_char

    def sequences_to_tags(self, sequences): # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
      result = []
      for sequence in sequences: 
          tag = []
          for pred in sequence: # 시퀀스로부터 예측값을 하나씩 꺼낸다.
              pred_index = np.argmax(pred) # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
              tag.append(index_to_ner[pred_index].replace("PAD", "O")) # 'PAD'는 'O'로 변경
          result.append(tag)
      return result

    # 에포크가 끝날 때마다 실행되는 함수
    def on_epoch_end(self, epoch, logs={}):

      # char Embedding을 사용하는 경우
      if self.use_char:
          # validation_data : 각단계마다 모델을 평가할 기준이 되는 x, y값을 저장
        X_test = self.validation_data[0]
        X_char_test = self.validation_data[1]
        y_test = self.validation_data[2]
        y_predicted = self.model.predict([X_test, X_char_test])

      else:
        X_test = self.validation_data[0]
        y_test = self.validation_data[1]
        y_predicted = self.model.predict([X_test])

      pred_tags = self.sequences_to_tags(y_predicted)
      test_tags = self.sequences_to_tags(y_test)

      score = f1_score(pred_tags, test_tags)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(test_tags, pred_tags))


      if score > self.value:
        print('f1_score improved from %f to %f, saving model to best_model.h5'%(self.value, score))
        # 가장 점수 높은 모델을 저장, 주소변경 필요
        self.model.save('drive/My Drive/Colab Notebooks/best_model.h5')
        self.value = score
      else:
        print('f1_score did not improve from %f'%(self.value))


In [ ]:

!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=20, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(50, activation="relu")))
crf = CRF(tag_size)
model.add(crf)

In [ ]:
model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])
history = model.fit(x_train, y_train, batch_size = 128, epochs = 25, validation_split = 0.1, verbose = 1, callbacks=[F1score(use_char=False)])

In [ ]:
from keras.models import load_model
bilstm_crf_model = load_model('drive/My Drive/Colab Notebooks/best_model.h5', custom_objects={'CRF':CRF,
                                                  'crf_loss':crf_loss,
                                                  'crf_viterbi_accuracy':crf_viterbi_accuracy})

In [ ]:
f1score = F1score(use_char=False)

y_predicted = bilstm_crf_model.predict([x_test])
pred_tags = f1score.sequences_to_tags(y_predicted)
test_tags = f1score.sequences_to_tags(y_test)
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

In [ ]:
new_encoded=[]
new_sentence='Mr. Heo said South Korea has become a worldwide leader'.lower().split()
for w in new_sentence:
    try:
      new_encoded.append(word_to_index.get(w,1))
    except KeyError:
      new_encoded.append(word_to_index['OOV'])
      # 모델이 모르는 단어에 대해서는 'OOV'의 인덱스인 1로 인코딩

new_padded = pad_sequences([new_encoded], padding="post", value=0, maxlen=max_len)

p = bilstm_crf_model.predict(np.array([new_padded[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("단어", "예측값"))
print(30 * "=")
for w, pred in zip(new_sentence, p[0]):
    print("{:15}: {:5}".format(w, index_to_ner[pred]))